# NLP cơ bản

Ở notebook này, chúng ta sẽ sử dụng truyện ngắn [_An Occurrence at Owl Creek Bridge_](https://en.wikipedia.org/wiki/An_Occurrence_at_Owl_Creek_Bridge) của Ambrose Bierce (1890). <br>Truyện này thuộc phạm vi công cộng; file text được lấy từ [Project Gutenberg](https://www.gutenberg.org/ebooks/375.txt.utf-8).

In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.8 MB 1.3 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.8 MB 1.3 MB/s eta 0:00:09
     ---- ----------------------------------- 1.3/12.8 MB 1.4 MB/s eta 0:00:09
     ----- ---------------------------------- 1.8/12.8 MB 1.6 MB/s eta 0:00:07
     ------ --------------------------------- 2.1/12.8 MB 1.6 MB/s eta 0:00:07
     -------- ------------------------------- 2.6/12.8 MB 1.8 MB/s eta 0:00:06
     --------- ------------------------------ 3.1/12.8 MB 1.9 MB/s eta 0:00:06
     ----------- ---------------------------- 3.7/12.8 MB 1.9 MB/s eta 0:00:05
     ------------- -------------------------- 4.5/12.8 MB 2.1 MB/s eta 0:

In [1]:
# CHẠY CELL NÀY để thực hiện các import tiêu chuẩn:
import spacy

nlp = spacy.load('en_core_web_sm')

**1. Tạo đối tượng Doc từ file `owlcreek.txt`**<br> 
> GỢI Ý: Dùng `with open('../TextFiles/owlcreek.txt') as f:`

In [2]:
# Nhập code ở đây:
with open("../TextFiles/owlcreek.txt") as f:
	doc = nlp.make_doc(f.read())

In [3]:
# Chạy cell này để xác minh là nó hoạt động:

doc[:36]

AN OCCURRENCE AT OWL CREEK BRIDGE

by Ambrose Bierce

I

A man stood upon a railroad bridge in northern Alabama, looking down
into the swift water twenty feet below.  

**2. File có chứa bao nhiêu token?**

In [4]:
num_tokens = len(doc)
num_tokens

4835

**3. File có bao nhiêu câu?**<br>GỢI Ý: Bạn sẽ cần tạo một list trước tiên!

In [5]:
import pypdf

with open("../TextFiles/owlcreek.txt", 'r', encoding = 'utf-8') as f:
	all_text = f.read()

doc = nlp(all_text)

# Đếm số lượng câu
num_sentences = len(list(doc.sents))
print(f"Số lượng câu trong file PDF: {num_sentences}")


Số lượng câu trong file PDF: 204


**4. In ra câu thứ ba trong tài liệu**<br> GỢI Ý:  Lập chỉ mục bắt đầu từ 0 và tiêu đề chính là câu đầu tiên.

In [6]:
print(list(doc.sents)[2])

A rope closely encircled his
neck.  


**5. Với mỗi token ở câu trên, hãy in ra `text`, tag `POS`, tag `dep` và `lemma`<br>
THỬ THÁCH: Sắp các giá trị thành hàng trong các cột và in ra kết quả.**

In [7]:
sentences = list(doc.sents)
third_sentence = sentences[2]
for token in third_sentence:
	print(f"{token.text:<15}{token.pos_:<10}{token.dep_:<15}{token.lemma_:<15}")

A              DET       det            a              
rope           NOUN      nsubj          rope           
closely        ADV       advmod         closely        
encircled      VERB      ROOT           encircle       
his            PRON      poss           his            

              SPACE     dep            
              
neck           NOUN      dobj           neck           
.              PUNCT     punct          .              
               SPACE     dep                           


In [ ]:
# GIẢI PHÁP THÔNG THƯỜNG:
sentences = list(doc.sents)
third_sentence = sentences[2]
for token in third_sentence:
	print(f"{token.text:<15}{token.pos_:<10}{token.dep_:<15}{token.lemma_:<15}")

A DET det a
man NOUN nsubj man
stood VERB ROOT stand
upon ADP prep upon
a DET det a
railroad NOUN compound railroad
bridge NOUN pobj bridge
in ADP prep in
northern ADJ amod northern
Alabama PROPN pobj alabama
, PUNCT punct ,
looking VERB advcl look
down PART prt down

 SPACE  

into ADP prep into
the DET det the
swift ADJ amod swift
water NOUN pobj water
twenty NUM nummod twenty
feet NOUN npadvmod foot
below ADV advmod below
. PUNCT punct .
  SPACE   


In [ ]:
# GIẢI PHÁP NÂNG CAO:



A               DET   det        a              
man             NOUN  nsubj      man            
stood           VERB  ROOT       stand          
upon            ADP   prep       upon           
a               DET   det        a              
railroad        NOUN  compound   railroad       
bridge          NOUN  pobj       bridge         
in              ADP   prep       in             
northern        ADJ   amod       northern       
Alabama         PROPN pobj       alabama        
,               PUNCT punct      ,              
looking         VERB  advcl      look           
down            PART  prt        down           

               SPACE            
              
into            ADP   prep       into           
the             DET   det        the            
swift           ADJ   amod       swift          
water           NOUN  pobj       water          
twenty          NUM   nummod     twenty         
feet            NOUN  npadvmod   foot           
below           ADV 

**6. Viết một matcher tên là 'Swimming' để tìm cả hai lần xuất hiện của cụm "swimming vigorously" trong text**<br> 
GỢI Ý: Nên kèm pattern `'IS_SPACE': True` giữa hai từ!

In [8]:
# Import thư viện Matcher:

from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

In [9]:
# Tạo một pattern và thêm nó vào matcher:

pattern = [
	{"LOWER": "swimming"},
	{"IS_SPACE": True, "OP": "*"},  # Cho phép khoảng trắng hoặc không có
	{"LOWER": "vigorously"}
]
matcher.add("Swimming", [pattern])
matches = matcher(doc)

print(f"Số lần xuất hiện của cụm 'swimming vigorously': {len(matches)}")
for match_id, start, end in matches:
	matched_span = doc[start:end]
	print(f"Vị trí: {start}-{end}, Cụm: {matched_span.text}")

Số lần xuất hiện của cụm 'swimming vigorously': 2
Vị trí: 1274-1277, Cụm: swimming
vigorously
Vị trí: 3609-3612, Cụm: swimming
vigorously


In [ ]:
# Tạo một list các matches gọi là "found_matches" và in ra list:




[(12881893835109366681, 1274, 1277), (12881893835109366681, 3607, 3610)]


**7. In ra text quanh mỗi match được tìm thấy**

In [10]:
print(f"Số lần xuất hiện của cụm 'swimming vigorously': {len(matches)}")
for match_id, start, end in matches:
	matched_span = doc[start:end]
	sentence = matched_span.sent  # Lấy câu chứa cụm từ
	print(f"\nCụm: {matched_span.text}")
	print(f"Câu: {sentence.text}")

Số lần xuất hiện của cụm 'swimming vigorously': 2

Cụm: swimming
vigorously
Câu: By diving I could evade the bullets and, swimming
vigorously, reach the bank, take to the woods and get away home.  

Cụm: swimming
vigorously
Câu: The hunted man saw all this over his shoulder; he was now swimming
vigorously with the current.  


By diving I could evade the bullets and, swimming
vigorously, reach the bank, take to the woods and get away home


over his shoulder; he was now swimming
vigorously with the current.  


**EXTRA CREDIT:<br>In ra *câu* chứa mỗi match được tìm thấy**

By diving I could evade the bullets and, swimming
vigorously, reach the bank, take to the woods and get away home.  


The hunted man saw all this over his shoulder; he was now swimming
vigorously with the current.  
